<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage - Evaluate model
</div>

In [3]:
import os
import sys
sys.path.append(os.path.abspath(".."))

In [47]:
import re
import os
import json
import requests
from dotenv import load_dotenv
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from process_cv.info_extract import extract_info

In [24]:
load_dotenv()
API_KEY = os.getenv("MISTRAL_API_KEY", "8h738jV32gjV9nO7l2nphveXhkhsKao5")
API_URL = "https://api.mistral.ai/v1/chat/completions"
MODEL_NAME = "mistral-medium"

## I. Rule-Based Matching

Hàm `matching_score(cv, job)` dùng để tính toán độ phù hợp (matching score) giữa hồ sơ ứng viên và mô tả công việc. Mỗi yếu tố được gán trọng số cụ thể, dựa trên các tiêu chí tuyển dụng phổ biến trong thực tế.


### Thành phần đánh giá độ phù hợp

| Thành phần         | Lấy từ (CV)            | Lấy từ (Job)                        | Mục đích                      | Trọng số (%) |
|--------------------|------------------------|-------------------------------------|-------------------------------|--------------|
| `skills`           | `cv["skills"]`         | `job["technologies_used"]`         | So khớp kỹ năng kỹ thuật      | 55%          |
| `years_of_exp`     | `cv["years_of_experience"]` | `job["experience_years"]`      | So sánh kinh nghiệm làm việc | 20%          |
| `job_title`        | `cv["job_title"]`      | `job["job_title"]`                  | So tiêu đề công việc          | 20%          |
| `cv_skills text`   | `cv["skills"]` (nối chuỗi) | `job["job_description"]`        | So mức độ tương đồng nội dung JD | 5%       |


**a. Kỹ năng (Skills) – **55%****

- Trích xuất danh sách kỹ năng từ `cv["skills"]` và `job["technologies_used"]`.
- Chuẩn hóa chuỗi (viết thường, bỏ phần trong ngoặc).
- Sử dụng hàm `fuzzy_skill_match()` để đếm số kỹ năng khớp (có thể khớp mờ).
- Tính điểm theo tỷ lệ kỹ năng khớp / tổng kỹ năng yêu cầu.

*Càng nhiều kỹ năng khớp thì điểm kỹ năng càng cao (tối đa 55 điểm).*


**b. Kinh nghiệm làm việc (Experience) – **20%****

- Trích xuất số năm yêu cầu từ `job["experience_years"]` (ví dụ: `"Từ 2 năm"`).
- Đọc số năm thực tế từ `cv["years_of_experience"]` (kiểu số nguyên).
- Nếu ứng viên có số năm bằng hoặc lớn hơn yêu cầu -> đạt tối đa 20 điểm.
- Nếu ít hơn -> tính điểm tỷ lệ theo phần trăm.

*Phù hợp với tiêu chí tuyển dụng yêu cầu số năm kinh nghiệm tối thiểu.*


**c. Tiêu đề công việc (Job Title) – **20%****

- So sánh `cv["job_title"]` với `job["job_title"]` bằng đo độ tương đồng chuỗi (`SequenceMatcher`).
- Nếu giống hoàn toàn -> 20 điểm.
- Giống tương đối (>= 60%) -> 13 điểm, và thấp hơn nữa (>= 40%) → 7 điểm.

*Đảm bảo ứng viên đang tìm kiếm đúng vị trí công việc.*


**d. Mức độ tương đồng nội dung mô tả công việc (JD Similarity) – **5%****

- Ghép nội dung JD từ `job["job_description"]`.
- Ghép kỹ năng của CV làm văn bản đại diện CV.
- Dùng `TF-IDF` + `cosine_similarity` để đo mức độ tương đồng nội dung.
- Nhân điểm tương đồng với trọng số 5%.

*Nhắm tới các khớp ẩn mà kỹ năng rời rạc không nêu rõ (VD: nội dung JD có "Figma", dù không liệt kê trong skills).*


**e. Tổng điểm**

Tổng điểm cuối cùng là tổng các phần:
- `match_score = skill_score + exp_score + title_score + jd_score`
- Trả về dạng dict để giải thích chi tiết từng phần chấm.


**f. Ví dụ kết quả đầu ra:**

```json
{
  "match_score": 42,
  "skill_score": 17.5,
  "exp_score": 20.0,
  "title_score": 0.0,
  "jd_score": 4.5
}


Ta sẽ chuẩn hóa đầu vào của bộ dữ liệu

In [ ]:
def normalize_skill(skill):
    return skill.lower().split('(')[0].strip()

def is_similar(a, b, threshold=0.8):
    return SequenceMatcher(None, a, b).ratio() >= threshold

def fuzzy_skill_match(cv_skills, job_skills, threshold=0.8):
    matched = 0
    for js in job_skills:
        for cs in cv_skills:
            if is_similar(js, cs, threshold):
                matched += 1
                break
    return matched

def extrac_years(job):
    try:
        for s in job.get("experience_years", []):
            if "năm" in s.lower():
                digits = [int(d) for d in s.split() if d.isdigit()]
                if digits:
                    return digits[0]
    except:
        pass
    return 0

In [45]:
def matching_score_rule(cv, job):
    # ----- Skills (60%) -----
    cv_skills = [normalize_skill(s) for s in cv.get("skills", [])]
    job_skills_raw = job.get("technologies_used", [])
    job_skills = []
    for s in job_skills_raw:
        job_skills += s.lower().replace("&", "and").split()
    job_skills = list(set([normalize_skill(s) for s in job_skills]))

    skill_score = 0
    if job_skills:
        matched = fuzzy_skill_match(cv_skills, job_skills)
        skill_score = (matched / len(job_skills)) * 60

    # ----- Experience (20%) -----
    try:
        required_years = extrac_years(job)
        actual_years = int(cv.get("years_of_experience", 0))
        exp_score = min(actual_years / required_years, 1) * 20 if required_years > 0 else 0
    except:
        exp_score = 0

    # ----- Job Title (15%) -----
    cv_title = (cv.get("job_title") or "").lower()
    job_title = (job.get("job_title") or "").lower()
    title_score = 0
    if cv_title and job_title:
        similarity = SequenceMatcher(None, cv_title, job_title).ratio()
        if similarity >= 0.9:
            title_score = 15
        elif similarity >= 0.6:
            title_score = 10
        elif similarity >= 0.4:
            title_score = 5

    # ----- JD Similarity (5%) -----
    jd_text = " ".join(job.get("job_description", []))
    cv_text = " ".join(cv_skills)
    # print(f"JD Text: {jd_text}")
    # print(f"CV Text: {cv_text}")
    try:
        vectorizer = TfidfVectorizer().fit([jd_text, cv_text])
        vectors = vectorizer.transform([jd_text, cv_text])
        similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
        jd_score = similarity * 5
    except:
        jd_score = 0

    # ----- Total -----
    total = round(skill_score + exp_score + title_score + jd_score)

    return {
        "match_score": total,
        "skill_score": round(skill_score, 2),
        "exp_score": round(exp_score, 2),
        "title_score": round(title_score, 2),
        "jd_score": round(jd_score, 2)
    }

In [46]:
# Example usage
cv_info = {
    "full_name": "HOANG HUU TUYEN",
    "email": "hoanghuutuyen06022004@gmail.com",
    "phone": "0392159388",
    "job_title": "UX/UI Designer (Game)",
    "education": [
        {
            "degree": "Information technology",
            "university": "Academy Of Cryptography Techniques"
        }
    ],
    "years_of_experience": "6",
    "experience": [],
    "skills": [
        "Java(Spring Boot)", "JavaScript(React)", "HTML", "CSS", "SQLServer", "MySQL",
        "MongoDB", "Spring Security", "Spring Cloud", "Kafka", "Redis", "AWS S3", "Brevo",
        "Problem-solving", "Teamwork", "Effective communication"
    ],
    "certifications": [],
    "languages": ["English"]
}

job_info = {
    "job_title": "UX/UI Designer (Game)",
    "company_name": "CÔNG TY CỔ PHẦN SUNTEK",
    "salary": "Thương lượng",
    "address": ["Thành phố Thủ Đức, Hồ Chí Minh"],
    "date_posted": ["Đăng 1 giờ trước"],
    "industry": ["Giải trí/ Game"],
    "company_size": ["25-99 Nhân viên"],
    "company_nationality": ["Thailand"],
    "experience_years": ["Từ 2 năm"],
    "position_level": ["Junior", "Middle"],
    "employment_type": ["In Office"],
    "contract_type": ["Fulltime"],
    "technologies_used": ["UX/UI Design", "HTML & CSS", "UI/UX"],
    "job_description": [
        "Trách nhiệm công việc",
        "1. General task",
        "Have aesthetic thinking, color coordination and layout...",
        "Can use one of the product design tools such as: Figma, Adobe Illustrator, Photoshop...",
        "Ability to take clear notes in design files.",
        "Ability to organize design documents scientifically.",
        "Design the interface of menus, buttons, tabs, pop-ups, and graphical user interface elements.",
        "Create user interface mockups and prototypes that clearly demonstrate how the website works and looks.",
        "Make unique designs that improve the user experience and match game themes."
    ]
}
match_result = matching_score_rule(cv_info, job_info)
print("Matching Score:", match_result)

Matching Score: {'match_score': 55, 'skill_score': 20.0, 'exp_score': 20, 'title_score': 15, 'jd_score': 0.0}


## II. LLM-Based Matching

In [38]:
prompt_template = """
        You are a professional career advisor. Based on the candidate's CV and the job description, analyze and return structured feedback:

        CV:
        {cv_json}

        Job Description:
        {job_json}

        Return a JSON response in this format:
        {{
        "match_score": 0-100,
        "missing_skills": ["skill1", "skill2", "..."],
        "recommendations": [
            {{
            "skill": "skill name",
            "course": "course name",
            "link": "course url"
            }}
        ],
        "summary": "Short advice summary (3-4 lines)"
        }}

        Only return valid JSON. Do not include explanation or code block markers.
    """

In [39]:
def matching_score_llm(cv_info: dict, job_info: dict) -> dict:
    prompt = prompt_template.format(
        cv_json=json.dumps(cv_info, ensure_ascii=False),
        job_json=json.dumps(job_info, ensure_ascii=False)
    )

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        content = result["choices"][0]["message"]["content"].strip()
        content = re.sub(r"^```(?:json)?|```$", "", content.strip())
        return json.loads(content)
    except Exception as e:
        return {
            "error": str(e),
            "raw_response": response.text if 'response' in locals() else None
        }


In [37]:

# Example usage
cv_info = {
    "full_name": "HOANG HUU TUYEN",
    "email": "hoanghuutuyen06022004@gmail.com",
    "phone": "0392159388",
    "job_title": "UX/UI Designer (Game)",
    "education": [
        {
            "degree": "Information technology",
            "university": "Academy Of Cryptography Techniques"
        }
    ],
    "years_of_experience": "6",
    "experience": [],
    "skills": [
        "Java(Spring Boot)", "JavaScript(React)", "HTML", "CSS", "SQLServer", "MySQL",
        "MongoDB", "Spring Security", "Spring Cloud", "Kafka", "Redis", "AWS S3", "Brevo",
        "Problem-solving", "Teamwork", "Effective communication"
    ],
    "certifications": [],
    "languages": ["English"]
}

job_info = {
    "job_title": "UX/UI Designer (Game)",
    "company_name": "CÔNG TY CỔ PHẦN SUNTEK",
    "salary": "Thương lượng",
    "address": ["Thành phố Thủ Đức, Hồ Chí Minh"],
    "date_posted": ["Đăng 1 giờ trước"],
    "industry": ["Giải trí/ Game"],
    "company_size": ["25-99 Nhân viên"],
    "company_nationality": ["Thailand"],
    "experience_years": ["Từ 2 năm"],
    "position_level": ["Junior", "Middle"],
    "employment_type": ["In Office"],
    "contract_type": ["Fulltime"],
    "technologies_used": ["UX/UI Design", "HTML & CSS", "UI/UX"],
    "job_description": [
        "Trách nhiệm công việc",
        "1. General task",
        "Have aesthetic thinking, color coordination and layout...",
        "Can use one of the product design tools such as: Figma, Adobe Illustrator, Photoshop...",
        "Ability to take clear notes in design files.",
        "Ability to organize design documents scientifically.",
        "Design the interface of menus, buttons, tabs, pop-ups, and graphical user interface elements.",
        "Create user interface mockups and prototypes that clearly demonstrate how the website works and looks.",
        "Make unique designs that improve the user experience and match game themes."
    ]
}

match_result = matching_score_llm(cv_info, job_info)['match_score']
print("LLM Matching Score:", match_result)

LLM Matching Score: 30


## III. So sánh kết quả Human vs LLM

## IV. Phân tích độ chính xác (MSE, MAE)